# 材料名稱對照表 parser

將回收的表重新整理成可以直接發布的格式

In [1]:
from pathlib import Path

date = '20251023'

import pandas as pd

file_path = Path(rf"\\hp06316p\技術開發一部&二部\技術開發一部\Cell實驗專區\INX 給客戶, 供應商 PI, LC, Seal 代號參考\給各供應商_LCD_CF_PC_LC_材料代號_{date}.xlsx")
target_path = Path(rf"D:\Downloads\material_align_{date}.xlsx")

all_dfs = pd.read_excel(file_path, sheet_name=None)

In [2]:
def columns_separate(df):
    h = df.columns

    lst = list(
        filter(
            lambda x: not x[1].startswith('Unnamed'), 
            enumerate(h)
        )
    ) + [(len(h), None)]

    result = []
    for i, e in enumerate(lst):
        if e[1] is None: break
        result.append((e[1], e[0], lst[i+1][0]))
    return result

def table_separate(df):
    lst_df = []
    for item, start, end in columns_separate(df):
        tmp_df = df.iloc[:, start:end]
        tmp_df.columns = tmp_df.iloc[0,:]
        tmp_df = tmp_df.drop(index=0, axis=0)
        # print(tmp_df.columns)
        tmp_df = tmp_df.dropna(subset=['INX 品名'])
        tmp_df.insert(0, 'Item', item)
        lst_df.append(tmp_df)
    return lst_df

In [3]:
lcd_df = pd.concat(table_separate(all_dfs['LCD']), ignore_index=True)
cf_df = pd.concat(table_separate(all_dfs['CF']), ignore_index=True)
# cf_df = cf_df.rename(columns={'名稱': 'INX 品名'})
cf_df = cf_df[['Item', 'INX 品名', '統一代號', 'Vendor']]
result = pd.concat([lcd_df, cf_df], ignore_index=True)
result['mode'] = result['mode'].apply(lambda s: s.upper() if type(s) == str else None)
result['Type'] = result['Type'].apply(lambda s: s.upper() if type(s) == str else None)

In [4]:
result[result['Item']=='OC']

,Item,mode,Type,INX 品名,統一代號,Vendor
190,OC,None,None,SS3727-R24SLB,OC-J-2,JSR
191,OC,None,None,SS3727-R24SLBQ,OC-J-2,JSR
192,OC,None,None,JSS-925,OC-J-1,JSR
193,OC,None,None,V-259EH-417X10,OC-N-6,NSCM
194,OC,None,None,TG2288-SA1,OC-TT-2,Toray
195,OC,None,None,V-259EH-440X3,OC-N-W6,NSCM
196,OC,None,None,TG2235-SA1,OC-TT-4,Toray
197,OC,None,None,TG2269-SA2,OC-G-5,TG
198,OC,None,None,CT6000,OC-E-1,eCSC
199,OC,None,None,SS1084,OC-J-3,JSR


In [5]:
result.to_excel(target_path, index=False)

## Other Test

In [39]:
import duckdb

tmp_df = result[['統一代號', 'Vendor']][
    result['統一代號'].str.split('-').map(lambda a: len(a) == 3)
]
tmp_df['統一代號'] = tmp_df['統一代號'].str.split('-').map(lambda x: x[1])
tmp_df = duckdb.sql("SELECT DISTINCT * FROM tmp_df").df()

In [40]:
tmp_df.to_excel(r"D:\Downloads\tmp.xlsx")